__[A Baby Robot's Guide To Reinforcement Learning](https://towardsdatascience.com/tagged/baby-robot-guide)__

# Markov Decision Processes and Bellman Equations
## An Introduction to Reinforcement Learning: Part 2

<center><img src="images/header_part2.gif"/></center>

# Introduction

In the first part of this series on Reinforcement Learning we saw how the behaviour of an agent could be evaluated, to measure how well it performed on a given problem. 

The problem environment was split into a set of states and each of these states was given a value that reflected how good it was to be in that state. By selecting actions that moved Baby Robot in the direction of increasing state values, we were able to help him find his way to the exit of a simple grid level.

Additionally, we managed to do this using a limited amount of theory. Rather than defining the full Reinforcement Learning problem up front, we derived our equations as we needed them, starting simply and building on those we'd used before. As it turns out, the simple steps we've taken have already brought us close to defining the full Reinforcement Learning problem. In this part we'll take the final few steps needed to reach our goal.


___

___

This notebook accompanies the Towards Data Science article and is part of _[A Baby Robot's Guide To Reinforcement Learning](https://towardsdatascience.com/tagged/baby-robot-guide)_

## A note on the code used in this notebook

To keep this notebook relatively tidy, all of the main functionality is contained in Python classes, each of which is in its own file in the "lib" directory. 

The images in this notebook are created using the fantastic _[ipycanvas](https://ipycanvas.readthedocs.io/en/latest/ )_ library, which allows drawing to the browser canvas. Unfortunately these images aren't displayed by github's preview, so to view the images and play with the code you'll need to clone the github repository.

In [1]:
!pip install -U ipycanvas -q

In [2]:
import sys
sys.path.insert(0, './lib')

import os
import numpy as np
import random
from time import sleep, time
from enum import IntEnum

from ipywidgets import Layout
from ipywidgets import Play, IntProgress, HBox, VBox, link

from direction import Direction
from arrows import Arrows
from grid_level import GridLevel, setup_play_level
from robot_position import RobotPosition
from policy import Policy
from policy_evaluation import PolicyEvaluation

# Markov Processes (Markov Chains)

In the last part, Baby Robot managed to find his way out of a simple grid level. Unfortunately he still hasn't made his way back to his mum and instead finds himself in the new level shown below:

In [3]:
level_width = 3
level_height = 2
level = GridLevel( level_width,level_height,start=[0,0],end=[2,1],show_start_text=True,add_compass=True)
robot_position = RobotPosition(level,initial_sprite = 2)

splashes = [[0.0,1.0,0.5],
            [0.5,1.0,0.0]]

level.add_splashes( splashes )

level.canvases

MultiCanvas(height=132, sync_image_data=True, width=296)

While this is a pretty small level, and Baby Robot can see the exit, the roof has been leaking and so the floor is covered in puddles. Unfortunately, Baby Robot wasn't really designed to work in the wet. Puddles will slow him down and, even worse, can cause him to skid. As a result it's not guaranteed that he'll move in the desired direction if he starts in a square that contains a puddle. 

We do however still know all of the information about the system and know the probability that a puddle will cause a skid to occur:


* Small puddles have a 20% chance of skidding, so 80% of the time the chosen direction will be taken.


* Big puddles have a 60% chance of skidding, so the chosen direction will only be taken 40% of the time.


When a skid occurs it will cause Baby Robot to move in a direction different to the one that was chosen. The probability of this happening will be split evenly over the other possible directions. Additionally, we're initially only observing the environment. Baby Robot will be trying to make his way to the exit, but is at the mercy of the system dynamics. He has no control over the path that he takes and his next state will just be decided by the transition probability of his current state. At this stage there are no actions or rewards.

All of this can be summarised in the state transition diagram shown below.

<center><img src="images/state_transitions.png"/>
<br/><br/><i>State transition probabilities for the leaky grid level</i></center>



Initially, Baby Robot is undecided as to which way to move. Therefore, from the start state, he'll either move East to S1 or South to S3, with equal probability 0.5. 

State S1 contains a large puddle so, while Baby Robot would like to move to state S2, there's only a 0.4 chance of this actually happening. A more likely outcome is that he'll skid and will instead either end up in state S4 or back at the start. Since the probability of skidding is 0.6, and this is split equally between the 2 possible directions, the probability that he'll end up in S4 is 0.3 and the probability he'll go back to the start is also 0.3. 

State S4 also contains a large puddle, so will behave in the same way as state S1. From S4 Baby Robot would like to move to the exit, but there's only a 0.4 chance he'll actually move in this direction. Instead he may skid and either move to state S1 or to state S3, both with probability 0.3.

Similarly, if he begins by moving to state S3, which contains a small puddle, his chance of his next move taking him to state S4, where he'd like to go, is 0.8 and the chance of skidding back to the start is 0.2. S2 also contains a small puddle, so from here there's a 0.8 chance he'll move to the exit and a 0.2 chance he'll skid back to S1.

These state transition probabilities can also be expressed in a state transition matrix, as shown below. This matrix tells us how likely it is, if starting in any state, that you'll end up in any other state.

<center><img src="images/state_transition_table.png"/>
<br/><br/><i>State transition matrix for the leaky grid level</center></i>

Note that the probabilities of leaving any state always add up to one, as shown on the state transition diagram and also for each row in the state transition matrix. Also, when Baby Robot reaches the exit, this is the terminal state, from which he'll not move again. Therefore at the exit, the transition probability of staying at the exit is 1.0.

Beginning at the start of the level, we can follow a series of paths through the level until we reach the exit. Each of these paths represents an episode and each episode will follow a random trajectory that is defined by the system dynamics. Due to the randomness of the state transitions, the paths can be of any length.

So, each of the following episodes are valid paths through the level:


* <i>Start, S1, S4, Exit</i>
* <i>Start, S3, S4, S1, S2, Exit</i>
* <i>Start, S1, S2, S1, S4, Exit</i>


For the level given above, and its accompanying state transition diagram and matrix, we've described a _**Markov Process**_ (also known as a _**Markov Chain**_). This is a model of a stochastic process in which a sequence of events occur, with the probability of an event occurring being dependent only on the current state.

The complete set of states in our grid level make up the state space, which define all of the possible states in the system. Each of these states is independent of the previous states and therefore satisfies the _**Markov Property**_. All of the information about a state is contained in that state. As a result, we don't need to know what's happened in the past in order to predict what will happen in the future.

In practice its unlikely that a system's transition probabilities are known in advance. Therefore, to estimate how likely it is to move from one state to another, it's possible to observe multiple episodes and then take the average. This approach, of taking random samples to calculate estimates, is known as Monte Carlo Sampling and we'll examine this fully in a future article.

___

# Markov Reward Processes

The Markov process defines a state space and the transition probabilities of moving between those states. It doesn't specify which states are good states to be in, nor if it's good to move from one state to another. For this we need to add rewards to the system and move from a Markov process to a Markov Reward process.

In the first part of this series, we defined that each time Baby Robot moved from one square to the next he would receive a reward of -1. This encouraged him to take the shortest path through the level, since the longer he took the more negative his reward would be. Remember we're trying to maximise the total accumulated reward, so therefore having a large negative reward isn't good. 

In this leaky level we've specified that puddles will cause Baby Robot to slow down, with big puddles taking longer to traverse than small ones. With this in mind we can modify our rewards as follows:

* Small puddles take twice as long to move through as a standard grid square. Therefore, to account for this, a reward of -2 will be given for moving into a state with a small puddle.

* Similarly, large puddles take 4 times as long to move through, so a reward of -4 will be given for moving into a state that has a large puddle.

Here we've defined that the penalty for being in a puddle occurs when Baby Robot moves into a puddle, so the increased negative reward will be given when moving into the state with the puddle. It would also be perfectly reasonable to attribute the rewards when exiting the state containing the puddle, but we're trying to emphasise that it's not good to step into big puddles!

We've added these rewards to the state transition diagram, as shown below. The rewards obtained for each transition are shown in red, below the associated transition probability.

<center><img src="images/markov_reward_process.png"/>
<br/><br/><i>State reward process for the leaky level</center></i>

Note that we're still only observing the system and haven't yet introduced the concept of actions. Therefore, Baby Robot will move from one state to the next with a probability given by the current state's transition probabilities, except now he'll receive a reward when he makes each move.

From the Start state, Baby Robot can move in two possible directions, he can either move into the large puddle in state S1 and receive a reward of -4. Alternatively he can move to state S3, where there's a small puddle and so he'll be given a less negative reward of -2. Because  Baby Robot doesn't yet have the ability to choose which path he takes, the probability of moving to either S1 or S3 are equally likely and each will happen with a probability of 0.5.

Although Baby Robot can't yet influence the path he takes through the level, the addition of returns does allow us to evaluate how good it is to follow a particular path. From this, we can work out how good it is to be in any particular state, given the defined transition probabilities.

In part 1 we introduced the concept of a _**return**_. This is the total accumulated reward, starting from a time step 't'. For example, for the sample path {Start,S1,S4,Exit}, the return would be (-4 –4 –1) = -9. Note that although we have a 'Start' state in our system, representing where Baby Robot enters the grid level, it's not necessary to begin at this position when calculating a return. For example, at time step 't' we could say that Baby Robot was at state S1 and the return would then be calculated from this position until the end of the episode.

Additionally, since the random nature of state transitions means that, in theory, sequences could potentially be of infinite length, we added a discount factor to our return calculation. This prevents the return value from becoming excessively large and focuses attention on rewards that are obtained in the immediate future. At each increasing time step the reward is multiplied by an increasing power of the discount factor and the discount factor is chosen to be a value between 0 and 1. Therefore, the contribution of the reward obtained at each step decreases with time.

This gives us the discounted return formula:

<center><img src="images/equation_7.png" style="background-color:#FFFFFF"/>
<br/><i>Equation 1: discounted return</i></center>

where: 

* _**Gₜ**_ = the return (the total amount of reward accumulated over an episode, starting at time 't')
* _**Rₜ₊₁**_ = the reward obtained at time t+1
* _**γ**_ (gamma) = the discount factor, where 0 ≤ γ ≤ 1.



If we pick any state as the starting point (i.e. where we are at time 't') and follow a path to the end of the episode, adding the discounted reward at each time step, we get a return value that will be a very rough estimate of the state's value, describing how good it was to be in the initial state. If the return value is a large negative value then this isn't very good. 

Obviously, given the random nature of the paths through the system, the return value of a single episode isn't the most accurate estimate of a state's value. So, instead we take the expected value, which is effectively the average value over infinite episodes. This gives us the _**state value function v(s)**_:

<center><img src="images/state_value.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 2: state value function</center></i>

So, at time '<i>t</i>' if we start in state '<i>s</i>' and measure the expected return, we'll get the value of state '<i>s</i>', describing how good it is to be in that state.

Given that we haven't yet introduced actions into the mix, the value function may seem to be of little use. We could calculate how good it is to be in a particular state, but don't yet have any way to make use of this knowledge to let us take a path that would give the maximum return. However, by making a slight re-adjustment to this equation we can make it massively more powerful.

Look again at equation 1, the return value starting at time '<i>t</i>'. Currently this is expressed in terms of all future rewards. Although the rewards have been discounted, and so future rewards contribute less to the final return, if you want to calculate the return to full accuracy you in theory need to consider the rewards obtained at all future time steps. This makes things rather impractical, since for every state you'll need to consider every reward that could be obtained in the future.

But it's possible to change equation 1, from being expressed in terms of all future rewards, to instead make it recursive, expressing it in terms of the future return:

<center><img src="images/recursive_return.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 3: Return at time 't' given in terms of the immediate reward and the next return</center></i>

Now the return is given in terms of the immediate reward plus the discounted return that will be given at the next time step. By substituting this into equation 2, we can also express the state value in terms of the immediate reward and the future discounted return:

<center><img src="images/bellman_equation.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 4: The state value function expressed in terms of the immediate reward and the next return</center></i>

But <i>Gₜ₊₁</i> is just the return at the next time step, at which point we'll be at the next state which, by using equation 2 again, is just the value of the next state. So we can also express the state value in a recursive way, making it a function of the immediate reward and the value of the next state:

<center><img src="images/bellman_equation_2.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 5: The Bellman Equation</center></i>

This rearrangement of the state value function, decomposing it into the immediate reward <i>Rₜ₊₁</i> and the discounted value of the next state <i>γv(Sₜ₊₁)</i>, is known as the _**Bellman Equation**_, which is arguably the fundamental equation of Reinforcement Learning. Using this, the value of any state can be calculated simply by looking ahead to the next state as opposed to having to inspect every future state. Once the values of states are known, the best actions can be chosen to move between these states and ultimately the best policy may be found to solve the given problem.

We can now apply this to the states in our Markov Reward process, to calculate their values. To do this, we first need to express the Bellman Equation in a slightly different form, as shown below:

<center><img src="images/MRP_value.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 6: The Bellman Equation expressed in terms of transition and reward probabilities</center></i>

In equation 6 we've simply changed from expressing the state value function as an expectation, to instead averaging over each of the transitions that can happen in state '<i>s</i>'. 

There are a couple of points worth noting about this equation:

* The joint sum over all of the next states and the rewards is just a convenient way of writing that we're summing over all next states and all rewards. So in reality there are 2 sums: one for the next states, which uses the transition probability of moving to state <i>s′</i>, and one for the rewards, given the probability of receiving reward '<i>r</i>' when we start in state '<i>s</i>'.

* <i>p(s′,r|s)</i> is the probability of moving to state s′ and getting reward '<i>r</i>', given that we start in state 's'.

* These probabilities are multiplied by the immediate reward that is obtained and the discounted value of the next state.


In this format we can apply this equation to the states of our leaky grid level. So, for example, at the Start state, it has 2 possible transitions, each of which takes place with a probability of 0.5 and which give rewards of -4 and -2, when moving to states S1 and S3 respectively. So its state value is given by:

<center><img src="images/bellman_equation_start_state.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 7: the state value of the Start state</center></i>

Since this is a very simple level, that we know will ultimately end up at the exit state and terminate, we can simplify this further by setting the discount factor '<i>γ</i>' to 1. However, we're still left with the value of the Start state expressed in terms of the values of states 1 and 3, which we don't yet know.

In the first part of this series we saw how state values could be calculated using iterative policy evaluation. Starting with the initial assumption that each state had a value of zero, we could iteratively improve this estimate until we converged on the true state values. Although we didn't explicitly state it at the time, we were already using a partial form of the Bellman Equation to accomplish this.

Applying iterative policy evaluation to our Markov Rewards process version of the leaky level gives the state values shown below. These values were obtained after 32 iterations, when the maximum change in any state value had converged to less than 0.001 (although the values shown below are only to 1 decimal place):

<center><img src="images/MRP_state_values.png" style="background-color:#FFFFFF"/>
<br/><br/><i>State values for the MRP</center></i>

From the converged state values we can see that,beginning at the start state and moving to the exit will, on average, occur a reward penalty of -16.3. Similarly, if we've made it as far as state S2, then we can expect to incur a penalty of -4 to reach the exit.

So, now that we know all of the state values, we can plug the values for S1 and S3 into equation 7 above, to check we've got the correct value for the Start state:

<i><center>v(Start) = 0.5 * [-4 -11.9] + 0.5 * [-2 -14.8] = -16.35</center></i>

Similarly the value for S2 is given from the values of S1 and the Exit:

<i><center>v(S2) = 0.8 * [-1] + 0.2 * [-4 -11.9] = -0.8 -3.18 = -3.98</center></i>

The first 23 iterations of policy evaluations on the leaky grid level are shown below. It can be seen how the values convergence (only the first 23 out of the total 32 iterations are shown, since after this point the change in the state values occurs at a greater precision than the 1 decimal place we're displaying):

In [4]:
# the probability of moving from one state to the next
transition_probabilities = [
    [0.0, 0.5, 0.0, 0.5, 0.0, 0.0],     # Start
    [0.3, 0.0, 0.4, 0.0, 0.3, 0.0],     # S1
    [0.0, 0.2, 0.0, 0.0, 0.0, 0.8],     # S2
    [0.2, 0.0, 0.0, 0.0, 0.8, 0.0],     # S3
    [0.0, 0.3, 0.0, 0.3, 0.0, 0.4],     # S4
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]     # Exit

# the reward for moving to each of the states
state_rewards = [-1,-4,-2,-2,-4,-1]

# level details
level_width = 3
level_height = 2
splashes = [[0.0,1.0,0.5],
            [0.5,1.0,0.0]]

In [5]:
class State(IntEnum):
    Start, S1, S2, S3, S4, Exit = range(6)

class Puddle(IntEnum):
    Dry, Small, Large = range(3)    

In [6]:
level = GridLevel( level_width,level_height,start=[0,0],end=[2,1],show_start_text=False,add_compass=False,side_panel=True)
level.add_splashes( splashes )  

start_values = np.zeros((State.Exit+1))
iteration = 0

level.show_values(np.reshape(start_values, (2, -1)))   
level.canvases[3].clear_rect(200,0,400,400)   
level.side_panel_text(240,39,f"Iteration: {iteration}")

In [7]:
def get_state_value( state , start_values ):
  ''' calculate the value of the specified state using the supplied current state values '''

  # iterate over all possible next states
  state_value = 0
  for next_state in State:

    # add the reward for moving to the next state and the value of the next state
    state_value += transition_probabilities[state][next_state] * (state_rewards[next_state] + start_values[next_state])        

  return state_value 

start_values = np.zeros((State.Exit+1))
get_state_value( State.Start, start_values )  

-3.0

In [8]:
def standard_sweep(start_values):
  ''' perform a single sweep of all states 
    - this calculates the value of all states except the exit '''
    
  values = np.zeros((State.Exit+1))
  end = level.get_end()    
  
  for state in State:
    if state != State.Exit:
      values[state] = get_state_value( state, start_values )      

  return values 
  

start_values = np.zeros((State.Exit+1))
standard_sweep(start_values)    

array([-3. , -2.3, -1.6, -3.4, -2.2,  0. ])

In [9]:
def run_to_convergence(max_iterations = 100, threshold = 1e-3):
  ''' run until the values stop changing '''
  
  start_values = np.zeros((State.Exit+1))
  for n in range(max_iterations):

    # calculate the new value of each state based on the current values
    end_values = standard_sweep(start_values)

    # calculate the largest difference in the state values from the start to end of the iteration
    delta = np.max(np.abs(end_values - start_values))            

    # copy the end values into the start values           
    start_values = end_values    

    # test if the difference is less than the defined convergence threshold
    if delta < threshold:
        break                 

  # return the number of iterations taken to converge
  return n,start_values

In [10]:
iterations,state_values = run_to_convergence()
print(f"Convergence in {iterations} iterations")
print(f"Final state values {state_values}")

Convergence in 34 iterations
Final state values [-16.33962254 -11.85046135  -3.96995854 -14.83027662 -10.20377352
   0.        ]


In [11]:
def on_update(*args):        
  global start_values,iteration
  iteration += 1
  start_values = standard_sweep(start_values)  
  # level.show_values(start_values)   
  level.show_values(np.reshape(start_values, (2, -1)))   

  level.canvases[3].clear_rect(200,0,400,400)    

  level.side_panel_text(240,39,f"Iteration: {iteration}")
    
play, progress, layout = setup_play_level( level, on_update, max=30 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

<i><center>Iterative policy evaluation for the leaky grid level MRP</center></i>

From the calculated state values it's easy to see that the best route through the grid would be Start, S1, S2, Exit, to incur the lowest penalty. However, to follow this route we'd need to be able to choose which path to take, which, since we don't yet have actions in a MRP, isn't possible. For this we need to add actions and move to a Markov Decision Process (MDP).

___

# Markov Decision Processes (MDPs)

The Bellman Equation allows us to calculate state values by simply doing a one-step look ahead to the next state. Using these values we can then see which states are good to be in and which states should be avoided. However, in a Markov Reward Process we don't have any control over which states we move to. For this we need to introduce actions and instead move to a Markov Decision Process (MDP).

Actions allow decisions to be made. So it becomes possible to choose the state to move to and the reward to obtain. Although, in a stochastic system, you may not always get the chosen action or reward. Both the transition probability and the reward function now depend on the action. The action itself is selected by a _**policy**_. This defines which action should be chosen in any particular state.

By making a very slight adjustment to the Bellman Equation, we can modify it to take account of the policy:

<center><img src="images/bellman_policy_equation.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 8: The Bellman equation for policy π</center></i>

Equation 8 is such a subtle change to the standard Bellman equation from equation 5, that it would be easy to miss the change. The difference is that now the value of both state '<i>s</i>' and the next state <i>Sₜ₊₁</i>, where we'll move to at the next time step, are expressed in terms of the policy π. This means that if we are in state '<i>s</i>' at time '<i>t</i>', we'll choose an action given by the policy '<i>π</i>' and we'll continue to select actions according to this policy in all future states.

As we did for the MRP, we can change this equation into a usable form by expressing it in terms of the probabilities of taking a particular action '<i>a</i>', ending up in a successor state <i>s′</i> and receiving a reward '<i>r</i>', summed over all of these quantities:

<center><img src="images/bellman_policy.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 9: State value under policy π</center></i>

This is identical to equation 6, where we expressed the Bellman Equation in terms of transition and reward probabilities for the MRP, except now actions have been added. 

So the value of state '<i>s</i>' when following policy '<i>π</i>' is equal to:
    
* The sum over all actions that can be taken in a particular state. 


* Each action has a probability of occurring that is governed by the policy. So <i>π(a|s)</i> is the probability that action '<i>a</i>' will be taken given that we're in state '<i>s</i>'. Summing over these probabilities is effectively taking the average of all the actions for the state.


* The reward obtained for taking an action and the next state, where we end up after taking that action, are also stochastic, so we take the average of these by summing over the probabilities of them occurring multiplied by the immediate reward obtained and the discounted value of the next state.

___

<i>In the first part of this series we only looked at deterministic actions. When an action was taken we knew exactly what the next state would be and the amount of reward we'd get for taking the action. Now, in the full form of the Bellman Equation, the next state and reward for an action are determined by a probability distribution, p(s′,r|s,a). This is the probability of moving to state s′ and getting reward 'r', given that we start in state 's' and take action 'a'.

This is shown below for the possible actions from state S4 of our sample level:</i>

<center><img src="images/stochastic_state_s4.png" style="background-color:#FFFFFF"/>
<br/><br/><i>State S4 actions, transition probabilities and rewards.</center></i>

<i>S4 is a state that contains a large puddle, so taking an action in this state has a large chance of causing a skid to occur. As a result the probability of ending up in the desired state, after taking the chosen action, is only 0.4. There's a 0.6 chance of skidding and instead ending up in a state that wasn't the original target. This probability is split equally over the other possible states.
    
So, for example, from S4's 3 possible actions, North, East and West, Baby Robot would obviously like to choose to move East and arrive at the exit. However, he's only got a 0.3 chance of actually reaching this target state and claiming the standard -1 reward for doing so. A more likely outcome is that he'll skid. In which case there's a 0.3 chance of ending up at S1, giving him a large negative reward of -4, or that he'll get a reward of -2 and end up at state S3, again with there being a 0.3 chance of this happening.</i>

___

# Q-Values

If you know the values of the returns that can be expected from taking each individual action it becomes a lot easier to find the best actions to take. Consequently, due to the large amount of knowledge that individual action values can give about a system, they've been assigned their own value letter and are referred to as _**Q-Values**_.

In equation 9, given above, the state value function is given by the sum over all actions, where the probability of taking each action is multiplied by the value of that action. Therefore, the right-hand sum, taken over the next states and rewards, represents the value of taking a particular action. We can separate this out, to give us the action-value function, when taking action '<i>a</i>' in state '<i>s</i>' under policy '<i>π</i>':

<center><img src="images/rl_p2_equation10.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 10: Action value function for policy π</center></i>

<i>
For state S4, shown in the diagram above, Baby Robot would like to take the action that leads him to the exit. Using equation 10 we can calculate the action-value for taking this action to see if this would be a good action to take. Since this is a finite MDP we'll use a discount factor of γ = 1. Additionally, since the exit is a terminal state by definition it will have a state value of zero. We haven't yet calculated the state value functions for S1 and S3, the other possible states where Baby Robot could end up if he skids, so for now we'll just represent these in their 'V' format.
</i>

<br>
<i>
This gives the following action-value for taking the East action in state S4:
</i>

<br>
<i>
<center>q(S4,East) = 0.4 * (-1) + 0.3 * (-4 + V(S1)) + 0.3 * (-2 +V(S3))</center>
</i>

<br>
<i>
In comparison, taking the 2 other possible actions in this state would give:
</i>

<br>
<i>
<center>q(S4,North) = 0.4 * (-4 + V(S1)) + 0.3 * (-2 + V(S3)) + 0.3 * (-1)</center>
</i>

<br>
<i>
<center>q(S4,West) = 0.4 * (-2 + V(S3)) + 0.3 * (-4 + V(S1)) + 0.3 * (-1)</center>
</i>


___

In [12]:
connection_matrix = [
# Start, S1, S2, S3, S4, Exit     
    [ 0,  1,  0,  1,  0,  0 ],     # Start
    [ 1,  0,  1,  0,  1,  0 ],     # S1
    [ 0,  1,  0,  0,  0,  1 ],     # S2
    [ 1,  0,  0,  0,  1,  0 ],     # S3
    [ 0,  1,  0,  1,  0,  1 ],     # S4
    [ 0,  0,  0,  0,  0,  1 ]]     # Exit

In [13]:
def get_puddle_size( s : State ):
    ''' get the size of the puddle in the supplied state '''
    grid_width = len(splashes[0])
    # convert from splash value to puddle enum
    return Puddle(splashes[s//grid_width][s % grid_width]*2)

get_puddle_size( State.S2 )

<Puddle.Small: 1>

In [14]:
def get_action_reward( s : State ):
    ''' get the reward for taking an action that moves to state 's' '''
    puddle_size = get_puddle_size( s )
    return -pow(2,puddle_size)

get_action_reward( State.Exit )    

-1

In [15]:
def get_target_states( s : State ):
    ''' get the list of possible target states from the specified state '''    
    connections = connection_matrix[s]  
    # convert the list of connections into an array of state indexes      
    target_indexes = np.nonzero(connections)[0]   
    # convert the indexes to state enums
    return np.array([*State],object)[target_indexes]

get_target_states( State.S4 )   

array([<State.S1: 1>, <State.S3: 3>, <State.Exit: 5>], dtype=object)

In [16]:
def get_π( s : State ):
    ''' given a state 's' return the possible actions and their probabilities - this is the policy 'π' '''        
    
    # get the set of possible target states
    target_states = get_target_states(s)

    # this policy makes all actions equally likely, so the probability of any action = 1/number of actions
    action_probability = 1 / target_states.shape[0]
    
    # create a structured array of targets and their probabilites
    actions = np.zeros(target_states.shape[0], dtype=[('target', State), ('probability', float)])
    actions['target'] = target_states
    actions['probability'] = action_probability   

    return actions

get_π( State.S4 )    

array([(<State.S1: 1>, 0.33333333), (<State.S3: 3>, 0.33333333),
       (<State.Exit: 5>, 0.33333333)],
      dtype=[('target', 'O'), ('probability', '<f8')])

In [17]:
def get_p( current_state : State, target_state : State ):
    ''' given the current and targe states (i.e. an action) return 
        the probalities of the next states and the associated rewards '''

    # get the set of possible next states and remove the target state
    # - this will leave the set of states we may end up in if a skid occurs
    possible_states = get_target_states(current_state)       
    possible_states = np.delete(possible_states, np.where(possible_states == target_state))
    number_possible_states = len(possible_states)
    
    # there's always going to be at least one next state so create a structured array for this
    next_states = np.zeros(1, dtype=[('next_state', State), ('reward', int), ('probability', float)])

    # if there's no puddle we'll not skid and therefore reach the target state with probability = 1.0
    target_probability = 1.0

    current_puddle_size = get_puddle_size( current_state )    
    if current_puddle_size == Puddle.Small: 
        # if the current state has a small puddle then we'll move to the target with probability 0.8        
        target_probability = 0.8    
    elif current_puddle_size == Puddle.Large: 
        # if the current state has a large puddle then we'll move to the target with probability 0.4        
        target_probability = 0.4
      

    # add the target state
    next_states[0] = (State(target_state), get_action_reward(target_state), target_probability)     

    # if there's a probability of skidding add all the other states where we can end up
    skid_probability = (1 - target_probability)
    if skid_probability > 0. and number_possible_states > 0:

        # the probability of skidding is spread equally over all the other possible successor states
        skid_probability /= number_possible_states

        # add any possible states where we may end up if we skid    
        for skid_state in possible_states:
            next_states = np.append(next_states, 
                                    np.array([(State(skid_state), get_action_reward(skid_state), skid_probability)],
                                    dtype=next_states.dtype))

    return next_states
    

get_p( State.S4, State.S1 )

array([(<State.S1: 1>, -4, 0.4), (<State.S3: 3>, -2, 0.3),
       (<State.Exit: 5>, -1, 0.3)],
      dtype=[('next_state', 'O'), ('reward', '<i8'), ('probability', '<f8')])

In [18]:
def get_state_value( state , start_values ):
    ''' calculate the value of the specified state using the supplied current state values
      - this implements equation 9 '''

    # iterate over all possible actions for the state
    state_value = 0
    for action in get_π( state ):
        
        target_state = action[0]
        action_probability = action[1]
        
        # iterate over all possible next states and rewards for the action   
        q_value = 0
        for action_dynamics in get_p( state, target_state ):
            next_state = action_dynamics['next_state']
            reward = action_dynamics['reward']
            probability = action_dynamics['probability']

            q_value += probability * (reward + start_values[next_state])

        state_value += action_probability * q_value
    
    return state_value   


start_values = np.zeros((State.Exit+1))
get_state_value( State.S4, start_values )

-2.333333333333333

In [19]:
iterations,state_values = run_to_convergence()
print(f"Convergence in {iterations} iterations")
print(f"Final state values {state_values}")

Convergence in 63 iterations
Final state values [-25.19374305 -20.7283546  -12.86379092 -23.66081284 -17.12916203
   0.        ]


In [20]:
level_width = 3
level_height = 2

level = GridLevel( level_width,level_height,start=[0,0],end=[2,1],show_start_text=False,add_compass=False,side_panel=True)
level.add_splashes( splashes )  

start_values = np.zeros((State.Exit+1))
iteration = 0

level.show_values(np.reshape(start_values, (2, -1)))
level.canvases[3].clear_rect(200,0,400,400)   
level.side_panel_text(240,39,f"Iteration: {iteration}")

In [21]:
def on_update(*args):        
  global start_values,iteration
  iteration += 1
  start_values = standard_sweep(start_values)  
  level.show_values(np.reshape(start_values, (2, -1)))   

  level.canvases[3].clear_rect(200,0,400,400)    

  level.side_panel_text(240,39,f"Iteration: {iteration}")
    
play, progress, layout = setup_play_level( level, on_update, max=60 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

# The Bellman Optimality Equation

To navigate through our grid level in the shortest amount of time, or to gain the maximum reward in any Reinforcement Learning problem, in every state we want to select the action that gives the maximum expected return. In other words, since the policy is the thing that decides what actions to choose for each state, we want to find the _**optimal policy**_.

For any state, if we search through the set of all possible policies, we can find the action  that gives the maximum return. This is the _**optimal action-value function**_:

<center><img src="images/rl_p2_equation11.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 11: The optimal action-value function is given by taking the action with the maximum action-value over all policies</center></i>

Then, if for every state we know which is the optimal action, we can simply always choose the optimal action and get the _**optimal state-value function**_:

<center><img src="images/rl_p2_equation12.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 12: The optimal state-value function</center></i>

So the optimal value function for a state is given by selecting the action that gives the maximum return and then always selecting the optimal action in all future states.

As with the optimal action-value function, the optimal state-value function occurs when the state value is maximum over all policies. When this is true for all states, so that the value of every state is equal to or greater than the value it would have under any other policy, we've found the _**optimal policy**_.

By substituting the value of '<i>q</i>' from equation 10 into the optimal state-value function in equation 12, we get the _**Bellman Optimality Equation**_. The optimal value for a state is given by selecting the best action in that state and thereafter following the optimal policy to choose the best actions in all subsequent states:

<center><img src="images/rl_p2_equation10.png" style="background-color:#FFFFFF"/>
<br/><br/><i>Equation 13: The Bellman Optimality Equation defining the optimal state-value function</center></i>

In the next part we'll see how the Bellman Expectation and Optimality equations can be used in Policy and Value Iteration to locate the optimal policy.

For now, we can just act greedily with respect to the state values we found above for the stochastic policy to get the optimal actions. From these we can then calculate the state values for this optimal policy. The iterative policy evaluation for the optimal policy is shown below. This actually converges in 31 iterations and produces state values that are much better than those given by a random policy.

In [22]:
def get_π( s : State ):
    ''' redefine the function to return the optimal policy
      - given a state 's' return the optimal action with a probability of 1.0 '''        
    
    # define the optimal policy
    policy = { State.Start: State.S1, 
               State.S1: State.S2,
               State.S2: State.Exit,
               State.S3: State.S4,
               State.S4: State.Exit,
               State.Exit: State.Exit }             

    actions = np.array([(policy[s], 1.0)], dtype=[('target', State), ('probability', float)]) 
    return actions

get_π( State.S4 ) 

array([(<State.Exit: 5>, 1.)],
      dtype=[('target', 'O'), ('probability', '<f8')])

In [23]:
iterations,state_values = run_to_convergence()
print(f"Convergence in {iterations} iterations")
print(f"Final state values {state_values}")  

Convergence in 31 iterations
Final state values [-15.51801287 -11.51877405  -3.90360257 -14.50936986 -10.00792344
   0.        ]


In [24]:
level = GridLevel( level_width,level_height,start=[0,0],end=[2,1],show_start_text=False,add_compass=False,side_panel=True)
level.add_splashes( splashes )  

start_values = np.zeros((State.Exit+1))
iteration = 0

policy = Policy(level)
directions = policy.get_directions(np.reshape(start_values, (2, -1)))
level.show_directions(directions)
level.show_values(np.reshape(start_values, (2, -1)))

level.canvases[3].clear_rect(200,0,400,400)   
level.side_panel_text(240,39,f"Iteration: {iteration}")

In [25]:
def on_update(*args):        
  global start_values,iteration
  iteration += 1
  start_values = standard_sweep(start_values)  
   
  directions = policy.get_directions(np.reshape(start_values, (2, -1)))
  level.show_directions(directions)  
  level.show_values(np.reshape(start_values, (2, -1)))  

  level.canvases[3].clear_rect(200,0,400,400)    
  level.side_panel_text(240,39,f"Iteration: {iteration}")
    
play, progress, layout = setup_play_level( level, on_update, max=30 )
VBox((level.canvases, HBox((play, progress))),layout=layout) 

<center><i>Iterative policy evaluation for the optimal policy on the leaky grid level MDP</i></center>

Using this optimal policy, Baby Robot can now navigate through this simple leaky level and, while he may still skid when he encounters a puddle, he will reach the exit in the shortest possible time.

One such path through the level is shown below. A skid occurs when the actual state that Baby Robot ends up in doesn't match with his desired, target, state.

In [26]:
def get_current_state( x, y ):
    ''' given a grid coordinate return the current states '''    
    return State( x + y*3 ) 


def get_next_state( current_state ):
    ''' given the current state return the target state and the actual next states and the reward '''
     
    target_state = get_π(current_state)[0]['target']

    # get the possible next states given the action
    action_dynamics = get_p( current_state, target_state )

    # choose one of the possible next states using the given probabilities
    actual_target = random.choices( population=action_dynamics, weights=action_dynamics['probability'], k=1 )[0]

    return target_state, actual_target['next_state'], actual_target['reward'] 
  
def get_direction( current_state : State, next_state: State ):

    y1 = current_state // 3
    x1 = current_state - (y1*3)
    y2 = next_state // 3
    x2 = next_state - (y2*3)   

    if x2 > x1: return Direction.East
    if y2 > y1: return Direction.South
    if x2 < x1: return Direction.West
    if y2 < y1: return Direction.North    

    return Direction.Stay  

In [28]:
level = GridLevel( level_width,level_height,start=[0,0],end=[2,1],show_start_text=True,add_compass=False,side_panel=True)
level.add_splashes( splashes ) 
robot_position = RobotPosition(level)

last_direction = None
total_reward = 0
iteration = 0
frame_count = 0

level.canvases[3].clear_rect(200,0,400,400)   
current_state = get_current_state(0, 0)
level.side_panel_text(204,24,f"Reward: {total_reward:<7}")  
level.side_panel_text(204,40,f"Current: {current_state.name:<7}")  

In [29]:
def on_update(*args):  
  global frame_count, last_direction, total_reward, iteration
  
  # keep baby robot moving in the last direction until he reaches the next grid square
  if ((robot_position.x%robot_position.robot_size == level.padding) 
  and (robot_position.y%robot_position.robot_size == level.padding)):
      
    x,y = robot_position.get_cell_position() 
    current_state = get_current_state(x,y)
    target_state, next_state, reward = get_next_state( current_state )
    total_reward += reward
    iteration += 1
#     frame_count += 1  

    if current_state != State.Exit:
      last_direction = get_direction( current_state, next_state )
   
      level.canvases[3].clear_rect(200,0,400,400)    
      level.side_panel_text(204,24,f"Reward: {total_reward:<7}",align='left')  
      level.side_panel_text(204,40,f"Current: {current_state.name:<7}",align='left')  
      level.side_panel_text(204,56,f"Target: {target_state.name:<7}",align='left')  
      level.side_panel_text(204,72,f"Actual: {next_state.name:<7}",align='left')  
      
    else:
      last_direction = None     
  
  if last_direction is not None:
    robot_position.partial_move( last_direction ) 
    frame_count += 1  
    
play, progress, layout = setup_play_level( level, on_update, interval=100, max=120 )
VBox((level.canvases, HBox((play, progress))),layout=layout)   

<center><i>Baby Robot following the optimal policy through the leaky level</i></center>

# Summary

In the first part of this series we helped Baby Robot to escape from a simple grid level. We did this by evaluating the state values and then choosing a path based on these values, to let us find the shortest route through the level. Although we didn't know it at the time, the equations we were using were the Bellman equations and the system we were operating in could be described by a Markov Decision Process. We've now covered both of these.

A basic Markov process defines a set of states, each of which satisfies the Markov property, where no knowledge of the past is required. The transition probabilities define the probability of moving from one state to the next and, in a Reinforcement Learning problem, these are given by the current policy.

When rewards are added to the Markov process we, unsurprisingly, get a Markov Reward process. The Bellman Equations let us combine the transition probabilities with these rewards to calculate the value of each state, to give a measure of how good it is to be in each state.

Although in a Markov Reward process we can calculate the value of each state, it's still not possible to decide how to move through the states to maximise the rewards. To enable this, they need to be extended to allow actions to be taken and this is exactly what happens when we move to Markov Decision processes. When actions are added, a Markov Decision Process can be used to fully describe a Reinforcement Learning problem's environment, and how an agent acts within that environment.

# What's Next

We've now covered all of the basic theory that's required to describe a Reinforcement Learning problem. Additionally, we've already seen how we can use Policy Evaluation to calculate the state values from which, for simple problems, we were able to find the optimal policy. 

In the next part we'll extend this, to use the Bellman Equations that we've just looked at, in the Policy and Value Iteration algorithms. These will allow us to discover the optimal policy in more advanced settings.